In [1]:
import torch

print(torch.__version__)

is_cuda = torch.cuda.is_available()
#device = torch.device('cuda')
device = torch.device('cuda' if is_cuda else 'cpu')

2.0.1+cu118


In [19]:
from ultralytics import YOLO
import cv2

# Create a YOLOv8 model
model = YOLO("yolov8n.pt")


In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5  2023-7-19 Python-3.9.16 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [26]:
type(model)

models.common.AutoShape

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

path = '../ue/onnx/Content/Data'

img=cv2.imread(path +'/crowd.jpg' , cv2.IMREAD_COLOR)
img=cv2.resize(img, (640,640))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img/255. 

#print(len(img))

# Normalize the image
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img = (img - mean) / std

plt.imshow(img)
plt.show()

new_img=np.reshape(img,(1,3,640,640))
data = torch.Tensor(new_img)
data=data.to(device)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [4]:
prediction=model(data)
#result=torch.max(prediction,1)

In [5]:
# Iterate over each detection

result = prediction.cpu()
for i in range(result.shape[1]):
    detection = result[0][i]
    
    # Here we are assuming that the first 4 elements are the bounding box coordinates,
    # the 5th element is the objectness score, and the rest are the class scores.
    bbox = detection[:4]
    objectness = detection[4]
    class_scores = detection[5:]
    
    # Find the class with the highest score
    class_id = np.argmax(class_scores, axis=0)
    class_score = class_scores[class_id]

    if class_score > 0.85 and class_id == 0:
        # Write to file, draw bounding boxes, etc.
        # Note that the bbox coordinates will depend on how they are represented in the output.
        # For example, they could be [center_x, center_y, width, height] or [x1, y1, x2, y2].
        #f.write(f"Class: {class_id}, BBox: {bbox}\n")

        # Draw the bounding box
        # Note that we are assuming the bbox coordinates are in the format [center_x, center_y, width, height].
        # Depending on the model, this might be different (e.g., [x1, y1, x2, y2]).
        # 결과가 점 두개 일 경우
        x1, y1 = int(bbox[0] - bbox[2] / 2), int(bbox[1] - bbox[3] / 2)
        x2, y2 = int(bbox[0] + bbox[2] / 2), int(bbox[1] + bbox[3] / 2)
        #cv2.imshow("img", img)
        #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # 결과가 센터, 길이 일 경우
        cen_x = int(bbox[0])
        cen_y = int(bbox[1])
        cv2.circle(img, (cen_x,cen_y), 10,(0, 255, 0),2 )

        # Draw the class and confidence score
        label = f"Class: {class_id}, Confidence: {class_score:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        print(label)

# Display the frame
cv2.imshow("img", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

Class: 27, Confidence: 0.88
Class: 27, Confidence: 0.86
Class: 27, Confidence: 0.90
Class: 27, Confidence: 0.90
Class: 27, Confidence: 0.85
Class: 27, Confidence: 0.86


In [25]:
# Iterate over each detection

result = prediction.cpu()     

annotated_image = prediction.render()
annotated_image = annotated_image[0]

# Display the frame
cv2.imshow("img", annotated_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'Tensor' object has no attribute 'render'

In [69]:
class_probs = prediction[:, 5:]

# 각 원소에서 최대 값을 갖는 인덱스를 반환합니다.
_, max_labels = torch.max(class_probs, dim=1)


max_labels

tensor([[  714,  6380, 25124, 24586, 24612, 22198, 22836, 21240, 22835, 25164, 22756, 22332, 12941,  5756,  1359, 12689,  6395, 22875, 19026,  5877, 24367, 23477, 23517, 21670, 23476, 23310, 23350, 22875, 23809,  1842, 12885,  5921,  2469, 24977,  4160, 19055, 16706,   666,  6244, 18061, 19595, 14314, 19045, 22795,
          2788,    80,  3187, 12716, 16716, 17993, 16226,  2999, 24226, 16837,   321, 16837,  5512, 16881, 14393,   482, 17039, 24181, 25183, 22873, 25183, 24804, 23789, 22717, 22756,  5570,  4161, 14162,  4082, 21923, 23973, 25019, 22150, 23603,  4411,  1926, 25038, 22146, 23517, 21838, 20547]], device='cuda:0')

In [71]:
# 각 원소에서 최대 값을 가지는 인덱스를 찾습니다.
values, indices = torch.max(prediction, dim=1)
indices

tensor([[  719,  6385, 25129, 24591, 24617, 22203, 22841, 21245, 22840, 25169, 22761, 22337, 12946,  5761,  1364, 12694,  6400, 22880, 19031,  5882, 24372, 23482, 23522, 21675, 23481, 23315, 23355, 22880, 23814,  1847, 12890,  5926,  2474, 24982,  4165, 19060, 16711,   671,  6249, 18066, 19600, 14319, 19050, 22800,
          2793,    85,  3192, 12721, 16721, 17998, 16231,  3004, 24231, 16842,   326, 16842,  5517, 16886, 14398,   487, 17044, 24186, 25188, 22878, 25188, 24809, 23794, 22722, 22761,  5575,  4166, 14167,  4087, 21928, 23978, 25024, 22155, 23608,  4416,  1931, 25043, 22151, 23522, 21843, 20552]], device='cuda:0')

In [72]:
values

tensor([[6.37831e+02, 6.36785e+02, 5.85778e+02, 5.81735e+02, 5.06263e-02, 6.09588e-01, 7.77422e-02, 2.53802e-01, 1.07001e-02, 1.58225e-02, 3.48209e-02, 5.41660e-02, 3.05710e-02, 2.08762e-02, 1.46412e-01, 8.28573e-03, 3.16394e-02, 1.20245e-02, 2.33097e-01, 1.62382e-01, 4.67788e-01, 8.53654e-02, 3.24890e-02, 2.61373e-02,
         2.04276e-02, 1.62550e-02, 1.24819e-02, 2.23248e-02, 2.53569e-02, 9.42259e-02, 5.60028e-02, 6.61104e-02, 2.01514e-01, 6.24964e-02, 1.27137e-02, 6.98923e-02, 7.42446e-03, 4.54374e-01, 2.89391e-02, 4.99776e-02, 1.21343e-02, 2.50117e-02, 2.79804e-02, 4.63568e-01, 1.35347e-01, 1.52398e-02, 5.79036e-02, 2.27570e-02,
         2.38771e-02, 2.39566e-02, 1.87711e-02, 2.67193e-01, 8.02055e-02, 6.74271e-03, 8.22007e-02, 6.79692e-02, 5.84662e-02, 1.10616e-02, 8.35919e-03, 1.20875e-02, 1.24574e-02, 6.59492e-01, 2.06736e-02, 1.75645e-01, 7.53320e-02, 1.15753e-01, 1.34918e-01, 7.20743e-02, 1.62695e-02, 8.46506e-03, 1.88339e-01, 1.18081e-02,
         1.24907e-01, 6.35765e-03, 1.

In [41]:
prediction.size()

torch.Size([1, 25200, 85])

In [43]:
prediction.shape[0]

1

In [58]:
print(f"Model output shape: {prediction[0].shape}")
print(f"First detection output: {prediction[0][0]}")
len(prediction[0][0])

Model output shape: torch.Size([25200, 85])
First detection output: tensor([3.28576e+00, 4.77542e+00, 6.26843e+00, 1.06071e+01, 2.06258e-04, 1.44834e-01, 5.20893e-03, 2.79238e-02, 6.58255e-04, 2.83949e-03, 5.16544e-03, 1.97723e-03, 4.52193e-03, 2.10243e-03, 5.34555e-02, 1.01575e-03, 2.73818e-02, 1.42996e-03, 5.72227e-03, 8.06103e-03, 1.43057e-03, 1.25301e-03, 1.75629e-03, 8.48027e-04,
        1.91761e-03, 4.06064e-04, 2.30410e-04, 9.22637e-04, 2.31330e-03, 2.50930e-03, 2.13644e-02, 4.91720e-03, 7.16225e-03, 1.99889e-03, 2.68782e-03, 2.35963e-03, 4.33695e-04, 3.89315e-02, 2.43829e-02, 5.36547e-03, 2.27640e-03, 3.45344e-03, 1.63725e-03, 6.34735e-02, 1.94032e-02, 1.86454e-03, 8.23850e-03, 1.63037e-03,
        2.71972e-03, 1.25967e-03, 1.57429e-03, 1.19329e-03, 1.94779e-03, 3.38830e-04, 1.42421e-03, 1.47297e-03, 1.70547e-03, 4.42052e-04, 1.92384e-03, 1.42710e-03, 7.17107e-04, 3.09673e-02, 1.09702e-03, 1.97538e-02, 8.12152e-04, 3.34626e-03, 1.52669e-03, 3.47274e-03, 9.72056e-04, 9.86149e-04

85

In [59]:
prediction[0]['labels'][0]

IndexError: too many indices for tensor of dimension 2

In [56]:
_, class_indices = torch.max(prediction[..., 5:], dim=-1)
class_indices

tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')

In [55]:
prediction[0]

AttributeError: 'Tensor' object has no attribute 'info'

In [46]:
threshold = 0.85

for i in range(prediction[0]['boxes'].shape[0]):
    # 'person' 클래스의 인덱스는 1입니다.
    if prediction[0]['scores'][i] > threshold and prediction[0]['labels'][i] == 1:
        box = prediction[0]['boxes'][i].cpu().numpy().astype(int)

        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        score = prediction[0]['scores'][i].item()

        # 점수(또는 confidence) 값을 화면에 표시합니다.
        cv2.putText(img, f"Confidence: {score:.2f}", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

IndexError: too many indices for tensor of dimension 2

In [68]:
prd_arr = prediction.cpu().detach().numpy()

prd_arr

result_class=np.argmax(prd_arr[:,5:],1)
result_class

85

In [60]:
max_prob, max_label = torch.max(prediction[0]['scores'], dim=-1)


IndexError: too many indices for tensor of dimension 2

----

In [29]:
import onnx

# Load the ONNX model
model = onnx.load("test.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))


graph torch_jit (
  %onnx::Cast_0[FLOAT, 1x3x640x640]
) initializers (
  %model.model.model.0.conv.weight[FLOAT, 32x3x6x6]
  %model.model.model.0.conv.bias[FLOAT, 32]
  %model.model.model.1.conv.weight[FLOAT, 64x32x3x3]
  %model.model.model.1.conv.bias[FLOAT, 64]
  %model.model.model.2.cv1.conv.weight[FLOAT, 32x64x1x1]
  %model.model.model.2.cv1.conv.bias[FLOAT, 32]
  %model.model.model.2.cv2.conv.weight[FLOAT, 32x64x1x1]
  %model.model.model.2.cv2.conv.bias[FLOAT, 32]
  %model.model.model.2.cv3.conv.weight[FLOAT, 64x64x1x1]
  %model.model.model.2.cv3.conv.bias[FLOAT, 64]
  %model.model.model.2.m.0.cv1.conv.weight[FLOAT, 32x32x1x1]
  %model.model.model.2.m.0.cv1.conv.bias[FLOAT, 32]
  %model.model.model.2.m.0.cv2.conv.weight[FLOAT, 32x32x3x3]
  %model.model.model.2.m.0.cv2.conv.bias[FLOAT, 32]
  %model.model.model.3.conv.weight[FLOAT, 128x64x3x3]
  %model.model.model.3.conv.bias[FLOAT, 128]
  %model.model.model.4.cv1.conv.weight[FLOAT, 64x128x1x1]
  %model.model.model.4.cv1.conv.bias[F

In [17]:
#!pip install onnxruntime

     ---------------------------------------- 6.7/6.7 MB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB ? eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB 5.3 MB/s eta 0:00:00


In [30]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

path = '../ue/onnx/Content/Data'

img=cv2.imread(path +'/crowd.jpg' , cv2.IMREAD_COLOR)
img=cv2.resize(img, (640,640))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img/255. 

#print(len(img))

plt.imshow(img)
plt.show()

new_img=np.reshape(img,(1,3,640,640))
data = torch.Tensor(new_img)
data=data.to(device)

In [31]:
import onnxruntime as ort

# Start an ONNX runtime session
ort_session = ort.InferenceSession('./test.onnx')

# Convert the PyTorch tensor to a numpy array
input_data = data.cpu().numpy()

# The input name should match with the input name in your ONNX model
input_name = ort_session.get_inputs()[0].name

# Run the model with the input data
outputs = ort_session.run(None, {input_name: input_data})

In [34]:
for detection in outputs:
    x1, y1, x2, y2, confidence, class_id = detection
    if confidence > 0.5:  # You can adjust this threshold
        # Draw the bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Display the class ID and confidence
        cv2.putText(img, f'{class_id}: {confidence}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

# Show the image with bounding boxes
cv2.imshow("Detections", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ValueError: not enough values to unpack (expected 6, got 1)

----

In [36]:
import cv2
import onnxruntime
import numpy as np
from PIL import Image
from torchvision import transforms

# ONNX 모델 로드
ort_session = onnxruntime.InferenceSession('./test.onnx')

# ONNX 런타임에서 입력 및 출력 이름 가져오기
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

In [43]:
path = '../ue/onnx/Content/Data'

img=cv2.imread(path +'/crowd.jpg' , cv2.IMREAD_COLOR)
img=cv2.resize(img, (640,640))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img/255. 

#print(len(img))

plt.imshow(img)
plt.show()

new_img=np.reshape(img,(1,3,640,640))
new_img = torch.Tensor(new_img)
new_img=new_img.to(device)

new_img_np = new_img.cpu().numpy()

# # 이미지를 PIL 이미지로 변환합니다.
# pil_image = Image.fromarray(frame)
# # 이미지를 모델 입력에 맞게 전처리 합니다.
# # 모델 입력 크기에 따라 변경해야 합니다.
# preprocess = transforms.Compose([
#     transforms.Resize((480, 640)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# img = preprocess(pil_image)
#img = img.unsqueeze(0).numpy()

In [44]:
# ONNX 모델 실행
# results = ort_session.run([output_name], {input_name: img})
results = ort_session.run([output_name], {input_name: new_img_np})
# 모델의 결과를 확인합니다.
print(f"Model output shape: {results[0].shape}")
print(f"First detection output: {results[0][0]}")

# Iterate over each detection
for detection in results[0]:
    if detection[4] > 0.85:  # If the confidence score is greater than 0.85

        # Draw the bounding box
        x1, y1, x2, y2 = map(int, detection[:4])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw the class and confidence score
        label = f"Class: {int(detection[5])}, Confidence: {detection[4]:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Show the image with bounding boxes
cv2.imshow("Detections", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Model output shape: (1, 25200, 85)
First detection output: [[     5.9947      3.6581       11.43 ...   0.0030171    0.001819     0.01476]
 [     13.108      3.4031      22.399 ...   0.0036255   0.0021441    0.012951]
 [     16.948       3.492      18.063 ...   0.0029782   0.0010759   0.0099158]
 ...
 [     565.58      596.84      126.54 ...   0.0065204   0.0016524   0.0027361]
 [     587.81       597.2      108.06 ...   0.0052627   0.0015716   0.0030394]
 [     609.71         605      122.03 ...    0.005091   0.0018545   0.0036594]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
import onnxruntime
import numpy as np
import cv2

# Load the ONNX model
onnx_session = onnxruntime.InferenceSession("test.onnx")

# Load and prepare image
path = '../ue/onnx/Content/Data'

image=cv2.imread(path +'/crowd.jpg' , cv2.IMREAD_COLOR)

orig_image = cv2.resize(image, (640, 640))  # Resize image to 640x640
image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)  # Convert color space from BGR to RGB
image = image / 255.0  # Normalize pixel values (range [0, 1])
image = np.transpose(image, (2, 0, 1))  # Change image shape from (H, W, C) to (C, H, W)
image = np.expand_dims(image, axis=0).astype(np.float32)  # Add batch dimension and convert to float32

# Run the model with the processed image
outputs = onnx_session.run(None, {'onnx::Cast_0': image})

# outputs is a list of all the model's output tensors
output = outputs[0]  # Get the first output

# output processing
boxes = output[0, :, :4]
confidences = output[0, :, 4]
class_probs = output[0, :, 5:]

# Get class IDs
class_ids = np.argmax(class_probs, axis=1)

# Thresholds
confidence_threshold = 0.5
class_threshold = 0.5

# Get the class probabilities for the detected class IDs
class_confidences = class_probs[np.arange(len(class_probs)), class_ids]

# Iterate over the detections
for box, confidence, class_confidence, class_id in zip(boxes, confidences, class_confidences, class_ids):
    if confidence > confidence_threshold and class_confidence > class_threshold and class_id == 0:
        x, y, w, h = box
        x, y, w, h = int(x * 640), int(y * 640), int(w * 640), int(h * 640)  # Scale box to image's size
        cv2.rectangle(orig_image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Draw bounding box

cv2.imshow("Detection", orig_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
